In [1]:
using Plots, LinearAlgebra, HDF5, Random, FastGaussQuadrature, LaTeXStrings, Statistics, SparseArrays

In [2]:
pyplot()
Plots.PyPlotBackend()

Plots.PyPlotBackend()

In [3]:
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/splines.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/poisson_solver_splines.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/bump_on_tail_distributions.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/sampling.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/time_marching.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/h5routines.jl")
include("/home/tobias/Documents/IPP/VlasovPoissonRBM/src/visualisation.jl");

In [4]:
fpath = "runs/BoT_Np5e4_k_010_050_np_10_T25_projections.h5"

μₛₐₘₚ = h5read(fpath, "parameters/mu_samp")
μₜᵣₐᵢₙ = h5read(fpath, "parameters/mu_train")

IP = IntegratorParameters(h5read(fpath, "parameters/dt"),
                          h5read(fpath, "parameters/n_t"), 
                          h5read(fpath, "parameters/n_s"),
                          h5read(fpath, "parameters/n_p"),
                          h5read(fpath, "parameters/n_h"),
                          h5read(fpath, "parameters/N_p")
                          )

S = PBSpline(h5read(fpath, "parameters/p"), IP.nₕ, 2.0*pi/μₛₐₘₚ[1])

Ψ = h5read(fpath, "projections/Psi");
Ψₑ = h5read(fpath, "projections/Psi_e");
Πₑ = sparse(h5read(fpath, "projections/Pi_e"));

In [5]:
# Reference draw
P₀ = Particles(h5read(fpath, "initial_condition/x_0"),
                h5read(fpath, "initial_condition/v_0"),
                h5read(fpath, "initial_condition/w") );

In [6]:
μₛₐₘₚ' 

1×5 Adjoint{Float64,Array{Float64,1}}:
 0.3  0.03  0.1  4.5  0.5

# Test

In [7]:
μₜᵣₐᵢₙ

10×5 Array{Float64,2}:
 0.1       0.03  0.1  4.5  0.5
 0.144444  0.03  0.1  4.5  0.5
 0.188889  0.03  0.1  4.5  0.5
 0.233333  0.03  0.1  4.5  0.5
 0.277778  0.03  0.1  4.5  0.5
 0.322222  0.03  0.1  4.5  0.5
 0.366667  0.03  0.1  4.5  0.5
 0.411111  0.03  0.1  4.5  0.5
 0.455556  0.03  0.1  4.5  0.5
 0.5       0.03  0.1  4.5  0.5

In [26]:
nₜₑₛₜ = 10
κₜₑₛₜ_ₘᵢₙ = 0.1; κₜₑₛₜ_ₘₐₓ = 0.5

μₜₑₛₜ = zeros(nₜₑₛₜ, 5)
for i in 1:nₜₑₛₜ
    μₜₑₛₜ[i,:] = [κₜₑₛₜ_ₘᵢₙ, μₛₐₘₚ[2], μₛₐₘₚ[3], μₛₐₘₚ[4], μₛₐₘₚ[5]]
end

λ = 0
for i in 1:nₜₑₛₜ
    if nₜₑₛₜ > 1
        μₜₑₛₜ[i,1] = rand(1)[1]*(κₜₑₛₜ_ₘₐₓ - κₜₑₛₜ_ₘᵢₙ) + κₜₑₛₜ_ₘᵢₙ
#         μₜₑₛₜ[i,1] = (1-λ)*κₜₑₛₜ_ₘᵢₙ + λ*κₜₑₛₜ_ₘₐₓ
#         λ += 1/(nₜₑₛₜ-1)
    end
end  

μₜₑₛₜ = μₜₑₛₜ[sortperm(μₜₑₛₜ[:, 1]), :]

10×5 Array{Float64,2}:
 0.166245  0.03  0.1  4.5  0.5
 0.173762  0.03  0.1  4.5  0.5
 0.215176  0.03  0.1  4.5  0.5
 0.26139   0.03  0.1  4.5  0.5
 0.271365  0.03  0.1  4.5  0.5
 0.284359  0.03  0.1  4.5  0.5
 0.308004  0.03  0.1  4.5  0.5
 0.380056  0.03  0.1  4.5  0.5
 0.394371  0.03  0.1  4.5  0.5
 0.460984  0.03  0.1  4.5  0.5

In [27]:
GC.gc()

In [28]:
IPₜₑₛₜ = IntegratorParameters(IP.dt, IP.nₜ, IP.nₜ+1, nₜₑₛₜ, S.nₕ, IP.Nₚ)
ICₜₑₛₜ = IntegratorCache(IPₜₑₛₜ);
K = stiffnessmatrix_PBSpline(S)
K_ = zero(K); K_ .= K; K_[S.nₕ,:] = ones(S.nₕ);

In [29]:
@time Rₜₑₛₜ = integrate_vp(P₀, S, μₜₑₛₜ, μₛₐₘₚ, K_, IPₜₑₛₜ, ICₜₑₛₜ;
                            given_phi = false, save = true);
# Xₜₑₛₜ = Rₜₑₛₜ.X
# Vₜₑₛₜ = Rₜₑₛₜ.V
# Φₜₑₛₜ = Rₜₑₛₜ.Φ;

parameter nb. 1
parameter nb. 2
parameter nb. 3
parameter nb. 4
parameter nb. 5
parameter nb. 6
parameter nb. 7
parameter nb. 8
parameter nb. 9
parameter nb. 10
336.042136 seconds (382.97 M allocations: 58.279 GiB, 2.59% gc time)


In [30]:
Φₜₑₛₜ = copy(Rₜₑₛₜ.Φ)
size(Φₜₑₛₜ)

(16, 2510)

# Reduced Model

In [31]:
k = size(Ψ)[2]
kₑ = size(Ψₑ)[2]
k, kₑ

(114, 409)

In [32]:
RIC = ReducedIntegratorCache(IPₜₑₛₜ, k, kₑ);

In [33]:
ΨᵀPₑ = Ψ' * Ψₑ * inv(Πₑ' * Ψₑ)
ΠₑᵀΨ = Πₑ' * Ψ;

In [34]:
@time Rᵣₘ = reduced_integrate_vp(P₀, Ψ, ΨᵀPₑ, ΠₑᵀΨ, S, μₜₑₛₜ, μₛₐₘₚ, K_, IPₜₑₛₜ, RIC;
                                   DEIM=true, given_phi = false, save = true);
# Xᵣₘ = Ψ * Rᵣₘ.Zₓ
# Vᵣₘ = Ψ * Rᵣₘ.Zᵥ
# Φᵣₘ = Rᵣₘ.Φ;

running parameter nb. 1 with chi = 0.554149335040845
running parameter nb. 2 with chi = 0.5792057398423942
running parameter nb. 3 with chi = 0.7172543343318054
running parameter nb. 4 with chi = 0.8712993492525841
running parameter nb. 5 with chi = 0.9045491877259534
running parameter nb. 6 with chi = 0.9478632254107022
running parameter nb. 7 with chi = 1.0266797271565917
running parameter nb. 8 with chi = 1.266854255991193
running parameter nb. 9 with chi = 1.314568578732566
running parameter nb. 10 with chi = 1.5366137805086344
160.179194 seconds (4.87 M allocations: 17.211 GiB, 0.18% gc time)


In [60]:
# Saving

In [36]:
h5open("runs/BoT_Np5e4_k_010_050_np_10_T25_DEIM.h5", "w") do file
    g = g_create(file, "parameters") # create a group
    g["N_p"] = IPₜₑₛₜ.Nₚ
    g["n_h"] = S.nₕ
    g["p"] = S.p
    g["dt"] = IPₜₑₛₜ.dt
    g["n_t"] = IPₜₑₛₜ.nₜ
    g["n_s"] = IPₜₑₛₜ.nₛ
    g["n_p"] = IPₜₑₛₜ.nₚ
    g["mu_samp"] = μₛₐₘₚ 
    g["mu_train"] = μₜᵣₐᵢₙ
    g["mu_test"] = μₜₑₛₜ
    
    f = g_create(file, "snapshots")
    f["X_test"] = Rₜₑₛₜ.X
    f["V_test"] = Rₜₑₛₜ.V
    f["Phi_test"] = Rₜₑₛₜ.Φ
    f["X_rm"] = Ψ * Rᵣₘ.Zₓ
    f["V_rm"] = Ψ * Rᵣₘ.Zᵥ
    f["Phi_rm"] = Rᵣₘ.Φ
end;

In [24]:
norm(Rᵣₘ.Φ - Φₜₑₛₜ)

0.0

In [25]:
norm(Rₜₑₛₜ.Φ - Φₜₑₛₜ)

0.0